In [2]:
import pandas as pd
import numpy as np
from math import sqrt
from numpy import concatenate
from matplotlib import pyplot as plt
from pandas import read_csv, DataFrame, concat
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU
import tensorflow as tf
from datetime import datetime

plt.rcParams['figure.figsize'] = (15, 5) # Boyutu ayarla

# Load the dataset
dataset = read_csv("iztech.csv", # veri setinin adı
                   parse_dates={'dt': ['Name']}, # tarih sütununu parse et
                   infer_datetime_format=True, # tarih formatını otomatik olarak belirle
                   index_col=0, # index olarak tarih sütununu kullan
                   na_values=['nan', '?']) #veri setini yükle
dataset.fillna(0, inplace=True)  # NaN değerleri 0 ile doldur

# Drop unnecessary columns
dataset.drop(columns=['WS101Max', 'WS101Min', 'WS101Count', 'WS 76', 'WS76Max', 'WS76Min', 'WS76Std', 'WS76Count', 'WD28Max', 'WD28Min',
                      'WD28Std', 'WD28Count', 'rho10', 'rho10Max', 'rho10Std'], inplace=True) # gereksiz sütunları sil

dataset.drop(columns=['WS101Std', 'WS30', 'WS30Max', 'WS30Min', 'WS30Std', 'WS30Count', 'WD74', 'WD74Max', 'WD74Min', 'WD74Std',
                      'WD74Count', 'WD28'], inplace=True) # gereksiz sütunları sil

dataset = dataset[['T3', 'P2', 'RH3', 'WS101']] # sütunları yeniden düzenle
values = dataset.values # veri setini numpy dizisine çevir

# Ensure all data is float
values = values.astype('float32') # veri setini float32 formatına çevir

# Normalize input features
scaler = MinMaxScaler(feature_range=(0, 1)) # veri setini 0-1 aralığına normalize et
scaled = scaler.fit_transform(values) # normalize edilmiş veri setini scaled değişkenine ata
scaled = pd.DataFrame(scaled) # scaled değişkenini pandas dataframe formatına çevir

# Create a function to prepare time series data
def create_ts_data(dataset, lookback=1, predicted_col=3): # veri setini zaman serisi veri setine çevir
    temp = dataset.copy() # veri setini kopyala
    temp["id"] = range(1, len(temp) + 1) # id sütunu ekle
    temp = temp.iloc[:-lookback, :] # lookback değerine göre veri setini düzenle
    temp.set_index('id', inplace=True) # id sütununu index olarak ata
    
    predicted_value = dataset.copy() # veri setini kopyala
    predicted_value = predicted_value.iloc[lookback:, predicted_col] # lookback değerine göre veri setini düzenle
    predicted_value.columns = ["Predicted"] # sütun adını değiştir
    predicted_value = pd.DataFrame(predicted_value) # veri setini pandas dataframe formatına çevir
    predicted_value["id"] = range(1, len(predicted_value) + 1) # id sütunu ekle
    predicted_value.set_index('id', inplace=True) # id sütununu index olarak ata
    
    final_df = pd.concat([temp, predicted_value], axis=1) # veri setlerini birleştir
    
    return final_df # veri setini döndür

# Create time series data with a lookback of 1
reframed_df = create_ts_data(scaled, 1, 3) # veri setini zaman serisi veri setine çevir
reframed_df.fillna(0, inplace=True) # NaN değerleri 0 ile doldur
reframed_df.columns = ['var1(t-1)', 'var2(t-1)', 'var3(t-1)', 'var4(t-1)', 'var5(t-1)'] # sütun adlarını değiştir

# Split data into train and test sets
values = reframed_df.values # veri setini numpy dizisine çevir
training_sample = int(len(dataset) * 0.7) # veri setinin %70'ini eğitim verisi olarak ata
train = values[:training_sample, :] # eğitim verisini ata
test = values[training_sample:, :] # test verisini ata

# Split data into input and output
train_X, train_y = train[:, :-1], train[:, -1] # eğitim verisini girdi ve çıktı olarak ayır
test_X, test_y = test[:, :-1], test[:, -1] # test verisini girdi ve çıktı olarak ayır

# Reshape input to be 3D [samples, time steps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1])) # eğitim verisini 3 boyutlu diziye çevir
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1])) # test verisini 3 boyutlu diziye çevir 

# Build and compile the GRU model
model_gru = Sequential() # modeli oluştur
model_gru.add(GRU(75, return_sequences=True, input_shape=(train_X.shape[1], train_X.shape[2]))) # GRU katmanı ekle
model_gru.add(GRU(units=30, return_sequences=True)) # GRU katmanı ekle
model_gru.add(GRU(units=30))    # GRU katmanı ekle
model_gru.add(Dense(units=1)) # Dense katmanı ekle
model_gru.compile(loss='mae', optimizer='adam') # modeli derle

# Display model summary
model_gru.summary() # model özetini göster

# Train the model
gru_history = model_gru.fit(train_X, train_y,  # eğitim verisini kullanarak modeli eğit
                            epochs=10, # 10 epoch eğit
                            validation_data=(test_X, test_y), # test verisini kullanarak modeli değerlendir
                            batch_size=64,  # batch size 64
                            shuffle=False # veri setini karıştırma
                            ) # modeli eğit

# Predictions
pred_y = model_gru.predict(test_X) # test verisini kullanarak tahmin yap
pred_y_1 = model_gru.predict(train_X) # eğitim verisini kullanarak tahmin yap

# Plot training and validation loss
plt.plot(gru_history.history['loss'], label='GRU train', color='brown') # eğitim verisi için loss değerini çiz
plt.plot(gru_history.history['val_loss'], label='GRU test', color='blue') # test verisi için loss değerini çiz
plt.legend() # legend ekle
plt.show() # grafiği göster

# Calculate evaluation metrics
MSE = mean_squared_error(test_y, pred_y) # MSE hesapla
R2 = r2_score(test_y, pred_y) # R2 hesapla
RMSE = sqrt(mean_squared_error(test_y, pred_y)) # RMSE hesapla
MAE = mean_absolute_error(test_y, pred_y) # MAE hesapla

print("MSE:", MSE) # MSE değerini yazdır
print("R2:", R2) # R2 değerini yazdır
print("RMSE:", RMSE) # RMSE değerini yazdır
print("MAE:", MAE) # MAE değerini yazdır

# Plot predicted vs actual test values
plt.plot(test_y, label='Actual') # gerçek değerleri çiz
plt.plot(pred_y, label='Predicted') # tahmin edilen değerleri çiz
plt.legend() # legend ekle
plt.show()  # grafiği göster

# Visualization over the full data
tra = np.concatenate([train_X, test_X]) # eğitim ve test verilerini birleştir
tes = np.concatenate([train_y, test_y]) # eğitim ve test verilerini birleştir
fp = model_gru.predict(tra) # birleştirilmiş verileri kullanarak tahmin yap
plt.plot(tes, label='Actual') # gerçek değerleri çiz
plt.plot(fp, label='Predicted') # tahmin edilen değerleri çiz
plt.legend() # legend ekle
plt.show() # grafiği göster
 
# Inverse normalizing
dataset_1 = dataset[['WS101']] # sadece WS101 sütununu al
dataset_1 = dataset_1[40421:57744] # veri setini 40421-57744 aralığındaki değerlerle sınırla
values_1 = dataset_1.values # veri setini numpy dizisine çevir 

# Ensure all data is float
values_1 = values_1.astype('float32') # veri setini float32 formatına çevir

# Normalize input features
scaler_1 = MinMaxScaler(feature_range=(0, 1)) # veri setini 0-1 aralığına normalize et
scaled_1 = scaler_1.fit_transform(values_1) # normalize edilmiş veri setini scaled_1 değişkenine ata
scaled_1 = pd.DataFrame(scaled_1) # scaled_1 değişkenini pandas dataframe formatına çevir

inversed_1 = scaler_1.inverse_transform(scaled_1) # normalize edilmiş veriyi gerçek değerlere çevir 
print(inversed_1) # gerçek değerleri yazdır
print(values_1) # gerçek değerleri yazdır 
print(scaled_1) # normalize edilmiş veriyi yazdır 

test_y_inv = scaler_1.inverse_transform([test_y]) # test verisini gerçek değerlere çevir
test_y_inv = test_y_inv.reshape(17323, 1) # boyutunu düzenle

pred_y_inv = scaler_1.inverse_transform(pred_y) # tahmin edilen değerleri gerçek değerlere çevir
pred_y_1_inv = scaler_1.inverse_transform(pred_y_1) # tahmin edilen değerleri gerçek değerlere çevir

MSE = mean_squared_error(test_y_inv, pred_y_inv) # MSE hesapla
R2 = r2_score(test_y_inv, pred_y_inv) # R2 hesapla
RMSE = sqrt(mean_squared_error(test_y_inv, pred_y_inv)) # RMSE hesapla
MAE = mean_absolute_error(test_y_inv, pred_y_inv) # MAE hesapla

print("MSE (Inverse Normalized):", MSE) # MSE değerini yazdır
print("R2 (Inverse Normalized):", R2) # R2 değerini yazdır
print("RMSE (Inverse Normalized):", RMSE) # RMSE değerini yazdır
print("MAE (Inverse Normalized):", MAE)     # MAE değerini yazdır


FileNotFoundError: [Errno 2] No such file or directory: 'iztech.csv'